In [1]:
import networkx as nx
import sys
from ruamel.yaml import YAML
import numpy as np
from abc import ABC, abstractmethod
from collections import namedtuple
from copy import deepcopy
from functools import reduce
from typing import Callable, Any
from itertools import accumulate

In [2]:
from systemflow.node import *

In [3]:
ci = CollectImage()

In [4]:
ci.inputs.host_parameters.bitdepth

'bit depth (n)'

In [5]:
ci.outputs.msg_fields.time

'acquisition time (s)'

In [6]:
ci.outputs.msg_fields

In [7]:
ci.outputs.msg_fields.__dict__.items()

dict_items([('image_data', 'image data (B)'), ('time', 'acquisition time (s)')])

In [8]:
list(ci.outputs.msg_fields.__dict__.keys())

['image_data', 'time']

In [9]:
ci.outputs.msg_fields

In [10]:
vc = collect_parameters([CollectImage()])

ci_host = Component("Image sensor",
                    [CollectImage(),],
                    parameters = {vc.resolution: (4000, 6000),
                     vc.bitdepth: 16,
                     vc.sample_rate: 1000.0,
                     vc.pixelenergy: 1e-3,})

In [11]:
msg0 = Message({}, {})

In [12]:
image_data, sensor_props = ci(msg0, ci_host)

In [13]:
image_data

Message(fields={'image data (B)': np.float64(48000000.0), 'acquisition time (s)': 0.0}, properties={'resolution (n,n)': (4000, 6000, 16), 'sample rate (Hz)': 1000.0})

In [14]:
ct = CollectTemperature()

In [15]:
vc = collect_parameters([CollectTemperature()])

In [16]:
list(vc.__dict__.keys())

['t_bitdepth', 't_samplerate', 't_sensor_power']

In [17]:
ct_host = Component("Thermocouple",
                    [CollectTemperature(),],
                    {vc.t_bitdepth: 16,
                     vc.t_samplerate: 1200,
                     vc.t_sensor_power: 1e-3,},)

In [18]:
temp_data, thermo_props = ct(msg0, ct_host)

In [19]:
dm = OverwriteMerge()

In [20]:
dm([msg0, msg0])

Message(fields={}, properties={})

In [21]:
msg1 = Message({"spiciness": 1000}, {"seasoning": "hot sauce"})
msg2 = Message({"spiciness": 2}, {"seasoning": "black pepper", "dish": "pasta"}) 

In [22]:
set(msg2.fields.keys()).union(set(msg2.properties.keys()))

{'dish', 'seasoning', 'spiciness'}

In [23]:
dm([msg1, msg2])

No merge provided for spiciness, taking first value
No merge provided for seasoning, taking first value


Message(fields={'spiciness': 1000}, properties={'dish': 'pasta', 'seasoning': 'hot sauce'})

In [24]:
class MaxSpicyMerge(Merge):
    def __init__(self):
        super().__init__({"spiciness": np.max,}, {"seasoning": lambda x: x[1]})
    


In [25]:
msm = MaxSpicyMerge()

In [26]:
msm([msg1, msg2])

Message(fields={'spiciness': np.int64(1000)}, properties={'dish': 'pasta', 'seasoning': 'black pepper'})

In [27]:
image_data.fields

{'image data (B)': np.float64(48000000.0), 'acquisition time (s)': 0.0}

In [28]:
temp_data.fields

{'temperature data (B)': 2.0, 'time (s)': 0.0}

In [29]:
dm([image_data, temp_data])

No merge provided for sample rate (Hz), taking first value


Message(fields={'temperature data (B)': 2.0, 'image data (B)': np.float64(48000000.0), 'acquisition time (s)': 0.0, 'time (s)': 0.0}, properties={'resolution (n,n)': (4000, 6000, 16), 'sample rate (Hz)': 1000.0})

In [30]:
class RateMerge(Merge):
    def __init__(self):
        super().__init__({"time": np.min,}, {"sample rate (Hz)": np.min})

In [31]:
rm = RateMerge()

In [32]:
rm([image_data, temp_data])

Message(fields={'temperature data (B)': 2.0, 'image data (B)': np.float64(48000000.0), 'acquisition time (s)': 0.0, 'time (s)': 0.0}, properties={'resolution (n,n)': (4000, 6000, 16), 'sample rate (Hz)': np.float64(1000.0)})

In [33]:
class DataRate(Mutate):
    def __init__(self, name: str = "DataRate"):
        #Input message fields
        #transform on any field with data (bytes - B)
        msg_fields = VarCollection(bytes = Regex(r"\(B\)"),)
    
        #Input message properties
        msg_properties = VarCollection()

        #Input host parameters
        host_parameters = VarCollection()
        
        inputs = MutationInputs(msg_fields, msg_properties, host_parameters)

        #Output message fields
        msg_fields = VarCollection()

        #Output message properties
        msg_properties = VarCollection()

        #Output host properties
        host_properties = VarCollection(total_data = "total data (B)",)

        outputs = MutationOutputs(msg_fields, msg_properties, host_properties)

        super().__init__(name, inputs, outputs)

    def transform(self, message: Message, component: Component):
        total_data = 0.0
        data_unit = self.inputs.msg_fields.bytes.str
        for (key, value) in message.fields.items():
            if bool(re.search(data_unit, key)) and key != "total data":
                total_data += value

        new_msg_fields = {self.outputs.host_properties.total_data: total_data}
        return new_msg_fields, {}, {}

In [34]:
class StorageRate(Mutate):
    def __init__(self, name: str = "StorageRate"):
        #Input message fields
        #transform on any field with data (bytes - B)
        msg_fields = VarCollection(total_data = "total data (B)",)
    
        #Input message properties
        msg_properties = VarCollection(sample_rate = "sample rate (Hz)",)

        #Input host parameters
        host_parameters = VarCollection()
        
        inputs = MutationInputs(msg_fields, msg_properties, host_parameters)

        #Output message fields
        msg_fields = VarCollection()

        #Output message properties
        msg_properties = VarCollection()

        #Output host properties
        host_properties = VarCollection(storage_rate = "data rate (B/s)",)

        outputs = MutationOutputs(msg_fields, msg_fields, host_properties)

        super().__init__(name, inputs, outputs)

    def transform(self, message: Message, component: Component):
        storage_rate = message.fields[self.inputs.msg_fields.total_data] * message.properties[self.inputs.msg_properties.sample_rate]
        new_host_properties = {self.outputs.host_properties.storage_rate: storage_rate}
        return {}, {}, new_host_properties

In [35]:
ms_host = Component("Merge/Store", [DataRate(), StorageRate()], {}, {}, RateMerge())

In [36]:
nodes = [ci_host, ct_host, ms_host]

In [37]:
[c.name for c in nodes]

['Image sensor', 'Thermocouple', 'Merge/Store']

In [38]:
links = [DefaultLink("Image -> Merge/Store",
                     "Image sensor",
                     "Merge/Store"),
         DefaultLink("Thermocouple -> Merge/Store", "Thermocouple", "Merge/Store"),]

In [39]:
links

In [40]:
ms_graph = ExecutionGraph("Sense/Merge/Store", nodes, links)

In [41]:
ms_graph

In [42]:
ms_graph.graph.edges

OutEdgeView([('Image sensor', 'Merge/Store'), ('Thermocouple', 'Merge/Store')])

In [43]:
ms_graph.graph.nodes["Merge/Store"]['ref']

In [44]:
ms_graph.root

'Merge/Store'

In [45]:
rp = ms_graph.get_predecessors(ms_graph.get_node(ms_graph.root))

In [46]:
rp

In [47]:
c1 = rp[0](ms_graph)

In [48]:
c1[1]

[]

In [49]:
c1[0].output_msg

Message(fields={'image data (B)': np.float64(48000000.0), 'acquisition time (s)': 0.0}, properties={'resolution (n,n)': (4000, 6000, 16), 'sample rate (Hz)': 1000.0})

In [50]:
dr = DataRate()

In [51]:
c1[0].properties

{'sensor power (W)': 24000.0}

In [52]:
c2 = rp[1](ms_graph)

In [53]:
c2[0].output_msg

Message(fields={'temperature data (B)': 2.0, 'time (s)': 0.0}, properties={'sample rate (Hz)': 1200})

In [54]:
c2[0].properties

{'thermocouple power (W)': 0.001}

In [55]:
dr.inputs.msg_fields.bytes

In [56]:
msg3 = c2[0].output_msg

In [57]:
list(msg3.fields.keys())

['temperature data (B)', 'time (s)']

In [58]:
bool(re.search(dr.inputs.msg_fields.bytes.str, list(msg3.fields.keys())[0]))

True

In [59]:
rn = ms_graph.get_node(ms_graph.root)

In [60]:
im = rn.merge([c1[0].output_msg, c2[0].output_msg])

In [61]:
im

Message(fields={'temperature data (B)': 2.0, 'image data (B)': np.float64(48000000.0), 'acquisition time (s)': 0.0, 'time (s)': 0.0}, properties={'resolution (n,n)': (4000, 6000, 16), 'sample rate (Hz)': np.float64(1000.0)})

In [62]:
rn(ms_graph)

(<systemflow.node.Component at 0x133147a60>,
 [(<systemflow.node.Component at 0x133146800>, []),
  (<systemflow.node.Component at 0x133145a80>, [])])

In [63]:
list(flatten(rn(ms_graph)))

In [64]:
g2 = ms_graph()

In [65]:
g2.iteration

1

In [66]:
vc = collect_parameters([Convolve(),])

In [67]:
list(vc.__dict__.keys())

['kernel', 'filters']

In [68]:

convolve_host = Component("Convolution", [Convolve(),], {vc.kernel: (3,3), vc.filters: 8}, {})

In [69]:
cnv = Convolve()

In [70]:
g2.nodes[0].name

'Merge/Store'

In [71]:
exp_msg = g2.nodes[0].output_msg

In [72]:
exp_msg.fields

{'temperature data (B)': 2.0,
 'image data (B)': np.float64(48000000.0),
 'acquisition time (s)': 0.0,
 'time (s)': 0.0,
 'total data (B)': np.float64(48000002.0)}

In [73]:
exp_msg.properties

{'resolution (n,n)': (4000, 6000, 16), 'sample rate (Hz)': np.float64(1000.0)}

In [74]:
message, props = cnv(exp_msg, convolve_host)

In [75]:
message

Message(fields={'temperature data (B)': 2.0, 'image data (B)': np.float64(48000000.0), 'acquisition time (s)': 0.0, 'time (s)': 0.0, 'total data (B)': np.float64(48000002.0), 'features (B)': np.int64(21301344)}, properties={'resolution (n,n)': (4000, 6000, 16), 'sample rate (Hz)': np.float64(1000.0)})

In [76]:
props

{'conv ops (n)': np.float64(191712096000.0)}

In [77]:
from systemflow.classifier import *

In [78]:
gc = GaussianClassifier(3.0)

In [79]:
gc.solve_reduction([900, 100], 0.9)

In [80]:
gc.error_matrix

array([[0.98055646, 0.17499203],
       [0.01944354, 0.82500797]])

In [81]:
1000 * 0.1

100.0

In [82]:
gcf = GaussianClassify()

In [83]:
classify_host = Component(["Classify"], [GaussianClassify(), DataRate(), ClassifiedStorageRate()], {"skill (1)": 3, "variance (1)": 1, "reduction (%)": 0.9}, {})

In [84]:
msg2, props2 = gcf(message, classify_host)

In [85]:
msg2.fields

{'temperature data (B)': 2.0,
 'image data (B)': np.float64(48000000.0),
 'acquisition time (s)': 0.0,
 'time (s)': 0.0,
 'total data (B)': np.float64(48000002.0),
 'features (B)': np.int64(21301344),
 'contingency (2x2)': array([[882,  17],
        [ 17,  82]])}

In [86]:
msg2.properties

{'resolution (n,n)': (4000, 6000, 16),
 'sample rate (Hz)': np.float64(1000.0),
 'error matrix (2p, 2p)': array([[0.98055646, 0.17499203],
        [0.01944354, 0.82500797]])}

In [87]:
props2

{}

In [88]:
ClassifiedStorageRate()(msg2, classify_host)

(Message(fields={'temperature data (B)': 2.0, 'image data (B)': np.float64(48000000.0), 'acquisition time (s)': 0.0, 'time (s)': 0.0, 'total data (B)': np.float64(48000002.0), 'features (B)': np.int64(21301344), 'contingency (2x2)': array([[882,  17],
        [ 17,  82]])}, properties={'resolution (n,n)': (4000, 6000, 16), 'sample rate (Hz)': np.float64(1000.0), 'error matrix (2p, 2p)': array([[0.98055646, 0.17499203],
        [0.01944354, 0.82500797]])}),
 {'storage rate (B/s)': array([  933289.89794967, 39600383.97291303])})

In [89]:
get_rejected(msg2.fields["contingency (2x2)"])

array([882,  17])

In [90]:
get_passed(msg2.fields["contingency (2x2)"])

array([17, 82])

In [91]:
list(collect_parameters([CollectImage(), CollectTemperature(), Convolve(), GaussianClassify(), DataRate(), ClassifiedStorageRate()]).__dict__.values())

['resolution (n,n)',
 'bit depth (n)',
 'sample rate (Hz)',
 'pixel energy (J)',
 'temperature bitdepth (n)',
 'sample rate (Hz)',
 'thermocouple power (W)',
 'kernel (n,n)',
 'filters (n)',
 'skill (1)',
 'variance (1)',
 'reduction (%)']

In [92]:
nodes = [Component("Image sensor",
                    [CollectImage(),],
                    {"resolution (n,n)": (6000, 4000),
                     "bit depth (n)": 8,
                     "sample rate (Hz)": 1000,
                     "pixel energy (J)": 1e-6,},
                     {}),
        Component("Thermocouple",
                    [CollectTemperature(),],
                    {"temperature bitdepth (n)": 8,
                     "sample rate (Hz)": 1200,
                     "thermocouple power (W)": 1e-3,},
                     {}),
        Component("Merge/Convolve",
                    [Convolve(),],
                    {"kernel (n,n)": (3,3), "filters (n)": 8},
                    {},
                    RateMerge()),
        Component("Classify",
                    [GaussianClassify(), DataRate(), ClassifiedStorageRate(),],
                    {"skill (1)": 3, "variance (1)": 1, "reduction (%)": 0.9},
                    {}),]

In [93]:
links = [DefaultLink("Image -> Merge/Convolve",
                     "Image sensor",
                     "Merge/Convolve"),
         DefaultLink("Thermocouple -> Merge/Convolve", 
                     "Thermocouple",
                     "Merge/Convolve"),
        DefaultLink("Merge/Convolve -> Classify",
                    "Merge/Convolve",
                    "Classify"),]

In [94]:
class ConvPower(Metric):
    def __init__(self):
        super().__init__("Convolution Power", 
                         {},
                         {},
                         {"Merge/Convolve": Convolve().outputs.host_properties.ops},)
        
    def metric(self, graph: ExecutionGraph):
        host_props = graph.get_all_node_properties()
        ops = host_props["Merge/Convolve"][Convolve().outputs.host_properties.ops]
        power = ops * 1e-12

        metrics = {"power": power,}
        return metrics
    

In [95]:
scc_graph = ExecutionGraph("Sense/Convolve/Classify", nodes, links, [ConvPower(),])

In [96]:
scc2 = scc_graph(True)

Executing on node  Classify
Executing on node  Merge/Convolve
Executing on node  Image sensor
Executing on node  Thermocouple


In [97]:
scc2.iteration

1

In [98]:
scc2.metric_values

[{'power': np.float64(0.191712096)}]

In [99]:
scc2.nodes[0].name

'Classify'

In [100]:
scc2.nodes[0].output_msg

Message(fields={'image data (B)': np.float64(24000000.0), 'acquisition time (s)': 0.0, 'time (s)': 0.0, 'temperature data (B)': 1.0, 'features (B)': np.int64(21301344), 'contingency (2x2)': array([[882,  17],
       [ 17,  82]]), 'total data (B)': np.float64(45301345.0)}, properties={'resolution (n,n)': (6000, 4000, 8), 'sample rate (Hz)': np.int64(1000), 'error matrix (2p, 2p)': array([[0.98055646, 0.17499203],
       [0.01944354, 0.82500797]])})

In [101]:
scc2.get_node("Classify").output_msg

Message(fields={'image data (B)': np.float64(24000000.0), 'acquisition time (s)': 0.0, 'time (s)': 0.0, 'temperature data (B)': 1.0, 'features (B)': np.int64(21301344), 'contingency (2x2)': array([[882,  17],
       [ 17,  82]]), 'total data (B)': np.float64(45301345.0)}, properties={'resolution (n,n)': (6000, 4000, 8), 'sample rate (Hz)': np.int64(1000), 'error matrix (2p, 2p)': array([[0.98055646, 0.17499203],
       [0.01944354, 0.82500797]])})

In [102]:
scc2.root_node.properties

{'storage rate (B/s)': array([  880818.45604992, 37373970.45294715])}

In [103]:
ap = scc2.get_all_node_properties()

In [104]:
scc2.get_node("Classify").output_msg.properties

{'resolution (n,n)': (6000, 4000, 8),
 'sample rate (Hz)': np.int64(1000),
 'error matrix (2p, 2p)': array([[0.98055646, 0.17499203],
        [0.01944354, 0.82500797]])}

In [105]:
ap

{'Classify': {'storage rate (B/s)': array([  880818.45604992, 37373970.45294715])},
 'Merge/Convolve': {'conv ops (n)': np.int64(191712096000)},
 'Image sensor': {'sensor power (W)': 24.0},
 'Thermocouple': {'thermocouple power (W)': 0.001}}

In [106]:
#setup experiment
#assume skill ~ 4 * log10(filters)

In [107]:
scc2.get_all_node_parameters()

{'Classify': {'skill (1)': 3, 'variance (1)': 1, 'reduction (%)': 0.9},
 'Merge/Convolve': {'kernel (n,n)': (3, 3), 'filters (n)': 8},
 'Image sensor': {'resolution (n,n)': (6000, 4000),
  'bit depth (n)': 8,
  'sample rate (Hz)': 1000,
  'pixel energy (J)': 1e-06},
 'Thermocouple': {'temperature bitdepth (n)': 8,
  'sample rate (Hz)': 1200,
  'thermocouple power (W)': 0.001}}

In [108]:
n_filters = np.arange(1,10)
skills = 4 * np.log10(n_filters)

In [109]:
skills

array([0.        , 1.20411998, 1.90848502, 2.40823997, 2.79588002,
       3.112605  , 3.38039216, 3.61235995, 3.81697004])

In [110]:
scc2.root_node.output_msg.fields

{'image data (B)': np.float64(24000000.0),
 'acquisition time (s)': 0.0,
 'time (s)': 0.0,
 'temperature data (B)': 1.0,
 'features (B)': np.int64(21301344),
 'contingency (2x2)': array([[882,  17],
        [ 17,  82]]),
 'total data (B)': np.float64(45301345.0)}

In [111]:
scc2.root_node.properties

{'storage rate (B/s)': array([  880818.45604992, 37373970.45294715])}

In [112]:
def sweep_filters(filters: int, exg: ExecutionGraph):
    new_skill = 4 * np.log10(filters)
    new_params = {"Classify": {"skill (1)": new_skill},
                  "Merge/Convolve": {"filters (n)": filters}}
    new_exg = exg.with_updated_parameters(new_params)()
    cont = new_exg.root_node.output_msg.fields["contingency (2x2)"]
    tn, fn = get_rejected(cont)
    fp, tp = get_passed(cont)
    f1 = (2 * tp) / (2 * tp + fp * fn)
    power = new_exg.metric_values[0]["power"]
    return power, f1

In [113]:
sweep_filters(8, scc_graph)

(np.float64(0.191712096), np.float64(0.6896551724137931))

In [114]:
sweep_filters(9, scc_graph)

(np.float64(0.215676108), np.float64(0.7896995708154506))

In [115]:
filters = np.arange(1, 20)

In [116]:
data = list(map(lambda x: sweep_filters(x, scc_graph), filters))

In [117]:
stacked = np.array(data)

In [118]:
import plotly.graph_objects as go

In [119]:
fig = go.Figure()

# Add Power trace
fig.add_trace(go.Scatter(
    x=filters,
    y=stacked[:,0],
    mode='lines',
    name='Power',
    line=dict(color='blue')
))

# Add F1 Score trace
fig.add_trace(go.Scatter(
    x=filters,
    y=stacked[:,1],
    mode='lines',
    name='F1 Score',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title_text='Power and F1 Score By Convolution Filters',
    xaxis_title='Filters',
    yaxis_title='Power (W) / Score',
    legend_title_text='Metrics',
    width = 800,
    height = 600
)

fig